In [3]:
import numpy as np
import pandas as pd
from haversine import haversine

In [2]:
df_demand = pd.read_csv(r"C:\Users\Jang\Desktop\project\demand_list.csv",encoding="cp949")
df_candi = pd.read_csv(r"C:\Users\Jang\Desktop\project\194_후보지.csv",encoding="cp949")

In [5]:
I = df_demand.index
J = df_candi.index

In [6]:
import math
import random
from pyscipopt import Model, quicksum, multidict

def kmedian(I,J,c,k):
    """kmedian -- minimize total cost of servicing customers from k facilities
    Parameters:
        - I: set of customers
        - J: set of potential facilities
        - c[i,j]: cost of servicing customer i from facility j
        - k: number of facilities to be used
    Returns a model, ready to be solved.
    """

    model = Model("k-median")
    x,y = {},{}

    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="B", name="x(%s,%s)"%(i,j))

    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == 1, "Assign(%s)"%i)
        for j in J:
            model.addCons(x[i,j] <= y[j], "Strong(%s,%s)"%(i,j))
    model.addCons(quicksum(y[j] for j in J) == k, "Facilities")

    model.setObjective(quicksum(c[i,j]*x[i,j] for i in I for j in J), "minimize")
    model.data = x,y

    return model


def distance(x1,y1,x2,y2):
    return haversine((x1,y1), (x2,y2))

x_demand = df_demand["위도"].apply(lambda x : x)    # positions of the points in the plane
y_demand = df_demand["경도"].apply(lambda x : x)  
x_candi = df_candi["위도"].apply(lambda x : x)       # positions of the points in the plane
y_candi = df_candi["경도"].apply(lambda x : x)  
c = {}

w = df_candi["target"]

for i in I:
    for j in J:
        c[i,j] = distance(x_demand[i],y_demand[i],x_candi[j],y_candi[j])* w[j]

In [7]:
random.seed(67)

k = 5
model = kmedian(I,J,c,k)
    # model.Params.Threads = 1
model.optimize()
EPS = 1.e-1
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]

print("Optimal value:",model.getObjVal())
print("Selected facilities:", facilities)
print("Edges:", edges)
print("max c:", max([c[i,j] for (i,j) in edges]))

Optimal value: 3.2786593608622288
Selected facilities: [57, 67, 135, 158, 170]
Edges: [(6, 57), (51, 57), (57, 57), (84, 57), (93, 57), (94, 57), (104, 57), (114, 57), (181, 57), (183, 57), (188, 57), (189, 57), (190, 57), (192, 57), (216, 57), (238, 57), (241, 57), (254, 57), (338, 57), (379, 57), (389, 57), (426, 57), (463, 57), (465, 57), (479, 57), (492, 57), (503, 57), (514, 57), (532, 57), (540, 57), (556, 57), (562, 57), (564, 57), (577, 57), (583, 57), (595, 57), (608, 57), (615, 57), (618, 57), (632, 57), (636, 57), (637, 57), (659, 57), (660, 57), (726, 57), (731, 57), (732, 57), (791, 57), (808, 57), (834, 57), (856, 57), (864, 57), (866, 57), (895, 57), (923, 57), (926, 57), (941, 57), (945, 57), (1032, 57), (1043, 57), (1055, 57), (1079, 57), (1091, 57), (1095, 57), (1116, 57), (1131, 57), (1189, 57), (1213, 57), (1224, 57), (1228, 57), (1272, 57), (1273, 57), (1288, 57), (1300, 57), (7, 67), (10, 67), (15, 67), (16, 67), (17, 67), (18, 67), (21, 67), (24, 67), (29, 67), (

In [9]:
df_candi.loc[[57, 67, 135, 158, 170],:].to_csv("k=5.csv",encoding="cp949",index=False)

In [10]:
random.seed(67)

k = 28
model = kmedian(I,J,c,k)
    # model.Params.Threads = 1
model.optimize()
EPS = 1.e-1
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]

print("Optimal value:",model.getObjVal())
print("Selected facilities:", facilities)
print("Edges:", edges)
print("max c:", max([c[i,j] for (i,j) in edges]))

Optimal value: 3.257949849853084
Selected facilities: [1, 6, 8, 18, 28, 40, 57, 67, 80, 110, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 156, 158, 170, 171]
Edges: [(443, 1), (1197, 6), (1294, 8), (48, 18), (186, 18), (218, 18), (323, 18), (563, 18), (758, 18), (873, 18), (1024, 18), (1104, 18), (1246, 18), (59, 28), (64, 40), (6, 57), (51, 57), (57, 57), (84, 57), (93, 57), (94, 57), (104, 57), (114, 57), (181, 57), (183, 57), (188, 57), (189, 57), (190, 57), (192, 57), (216, 57), (238, 57), (241, 57), (254, 57), (338, 57), (379, 57), (389, 57), (426, 57), (463, 57), (465, 57), (479, 57), (492, 57), (503, 57), (514, 57), (532, 57), (540, 57), (556, 57), (562, 57), (564, 57), (577, 57), (583, 57), (595, 57), (608, 57), (615, 57), (618, 57), (632, 57), (636, 57), (637, 57), (659, 57), (660, 57), (726, 57), (731, 57), (732, 57), (791, 57), (808, 57), (834, 57), (856, 57), (864, 57), (866, 57), (895, 57), (923, 57), (926, 57), (941, 57), (945, 57), (1032, 57), (1

In [13]:
df_candi.loc[facilities,:].to_csv("k=28.csv",encoding="cp949",index=False)